# Import

In [1]:
import os
import sys
import tarfile
import re
from six.moves import urllib
import urllib.request
import tensorflow as tf
import numpy as np
import _pickle
import re
import time

# Parameters

In [2]:
flags = tf.app.flags
Flags = flags.FLAGS

flags.DEFINE_integer('batch_size', 128, 'batch size')
flags.DEFINE_integer('batch_size_test', 256, 'test batch size')
flags.DEFINE_string('data_dir', '../../DataSet/Cifar/cifar10', 'directory for cifar10 dataset')
flags.DEFINE_boolean('use_fp16', False, 'train the model using fp16')

flags.DEFINE_string('checkpoint_dir', './train', 'directory where to read and write checkpoint files')
flags.DEFINE_string('train_dir', './train', 'Directory to write event logs and checkpoint')
flags.DEFINE_string('eval_dir', './eval', 'eval directory')
flags.DEFINE_string('test_dir',  './test', 'test directory')

flags.DEFINE_integer('max_steps', 1000000, 'number of batches to run')
flags.DEFINE_integer('log_device_placement', False, 'whether to log device placement')
flags.DEFINE_string('eval_data', 'test','Either test or train_eval ??')

flags.DEFINE_integer('num_examples', 10000, 'number of examples to run')
flags.DEFINE_boolean('run_once', False, 'whether to run eval only once')
flags.DEFINE_integer('eval_interval_secs', 60 * 5, 'how oftern to run the eval')

flags.DEFINE_integer('num_gpus', 1, 'how many gpus to use')




Train_Image_Size = 32

Num_Classes = 10
Num_Examples_Per_Epoch_For_Train = 50000
Num_Examples_Per_Epoch_For_Eval = 10000

Moving_Average_Decay = 0.9999
Num_Epochs_Per_Decay = 350
Learning_Rate_Decay_Factor = 0.1
Initial_Learning_Rate = 0.0001



Tower_Name = 'tower'
Data_Url = 'http://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz' 

datafile1 = os.path.join(Flags.data_dir, 'data_batch_1')
datafile2 = os.path.join(Flags.data_dir, 'data_batch_2')
datafile3 = os.path.join(Flags.data_dir, 'data_batch_3')
datafile4 = os.path.join(Flags.data_dir, 'data_batch_4')
datafile5 = os.path.join(Flags.data_dir, 'data_batch_5')
testfile = os.path.join(Flags.data_dir, 'test_batch')

tfdatafile1 = os.path.join(Flags.data_dir, 'data_batch_1.tfrecords')
tfdatafile2 = os.path.join(Flags.data_dir, 'data_batch_2.tfrecords')
tfdatafile3 = os.path.join(Flags.data_dir, 'data_batch_3.tfrecords')
tfdatafile4 = os.path.join(Flags.data_dir, 'data_batch_4.tfrecords')
tfdatafile5 = os.path.join(Flags.data_dir, 'data_batch_5.tfrecords')
tftestfile = os.path.join(Flags.data_dir, 'test_batch.tfrecords')


# Create TFRecord Files 

In [3]:
def get_np_images_and_labels_from_file(file):
    import pickle
    with open(file, 'rb') as f:
        dict = pickle.load(f, encoding='latin1')
    images = dict['data']
    labels = dict['labels']
    return images, labels

for dtfile in [datafile1, datafile2, datafile3, datafile4, datafile5, testfile]:
    tffile = dtfile + '.tfrecords'
    if os.path.exists(tffile):
        continue
    images, labels = get_np_images_and_labels_from_file(dtfile)
    writer = tf.python_io.TFRecordWriter(tffile)
    for i in range(images.shape[0]):
        image = images[i,:]
        label = labels[i]
        feature = {'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label])), 'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image.tostring()]))}
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        serialized = example.SerializeToString()
        writer.write(serialized)
        

# Input Pipeline from TFRecords

In [4]:
def trainfilepath(i):
    return os.path.join(Flags.data_dir, 'data_batch_') + str(i) + ".tfrecords"

trainfiles = [trainfilepath(i) for i in (1,2,3,4,5)]
testfiles = [os.path.join(Flags.data_dir, 'test_batch.tfrecords')]

In [5]:
def input_batches(num_threads=8, test=False):
    min_queue_examples = 5000
    batch_size = Flags.batch_size
    capacity = 3 * batch_size + min_queue_examples
    
    tffiles = None
    if test:
        tffiles = testfiles
        num_of_epochs = 1
        batch_size = Flags.batch_size_test
    else:
        tffiles = trainfiles
        
    filename_queue = tf.train.string_input_producer(tffiles, shuffle=True)
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)

    features = tf.parse_single_example(serialized_example, features = {'label':tf.FixedLenFeature(dtype=tf.int64, shape=[]), 'image':tf.FixedLenFeature(dtype=tf.string, shape=[])})

    image = tf.reshape(tf.decode_raw(features['image'], tf.uint8), [32,32,3])
    
    image = tf.cast(image, tf.float32)
    label = features['label']
    
#label = tf.cast(features['label'], tf.int32)

#    image = tf.random_crop(image, (Train_Image_Size, Train_Image_Size, 3))
    if False:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_brightness(image, max_delta=63)
        image = tf.image.random_contrast(image, lower=0.2, upper=1.8)
#    image = tf.image.per_image_standardization(image)

    image_batch, label_batch = tf.train.shuffle_batch([image, label], batch_size=batch_size, min_after_dequeue=min_queue_examples, capacity=capacity, num_threads=num_threads)

    if Flags.use_fp16:
        image_batch = tf.cast(image_batch, tf.float16)
#    label_batch = tf.cast(label_batch, tf.float16)
    return image_batch, label_batch

# Model, SqueezeNet

In [6]:
def firemodule(inputs, squeeze_depth, expand_depth, scope=None):
    with tf.variable_scope(scope, 'fire'):
        _squeezed = tf.layers.conv2d(inputs, squeeze_depth, (1,1), padding='SAME', activation=tf.nn.relu, name='squeezed')
        _expanded_1by1 = tf.layers.conv2d(_squeezed, expand_depth, (1, 1), padding='SAME', activation=tf.nn.relu, name='expanded_1by1')
        _expanded_3by3 = tf.layers.conv2d(_squeezed, expand_depth, (3, 3), padding='SAME', activation=tf.nn.relu, name='expanded_3by3')
        _expanded = tf.concat([_expanded_1by1, _expanded_3by3], axis=3)
        return _expanded
        

In [7]:
def squeeze_net(images, num_classes, reuse=False):
    with tf.variable_scope('squeezenet', reuse=reuse):
        net = tf.layers.conv2d(images, 96, (2,2), padding='SAME', name='conv1')
        print(net.name, net.get_shape())
        net = tf.layers.max_pooling2d(net, (2,2), strides=(2,2), padding='same', name='maxpool1')
        print(net.name, net.get_shape())
        net = firemodule(net, 16, 64, 'fire2')
        print(net.name, net.get_shape())
        net = firemodule(net, 16, 64, 'fire3')
        print(net.name, net.get_shape())
        net = firemodule(net, 32, 128, 'fire4')
        print(net.name, net.get_shape())
        net = tf.layers.max_pooling2d(net, (2,2), strides=(2,2), padding='same', name='maxpool4')
        print(net.name, net.get_shape())
        net = firemodule(net, 32, 128, 'fire5')
        print(net.name, net.get_shape())
        net = firemodule(net, 48, 192, 'fire6')
        print(net.name, net.get_shape())
        net = firemodule(net, 48, 192, 'fire7')
        print(net.name, net.get_shape())
        net = firemodule(net, 64, 256, 'fire8')
        print(net.name, net.get_shape())
        net = tf.layers.max_pooling2d(net, (2,2), strides=(2,2), padding='same', name='maxpool8')
        print(net.name, net.get_shape())
        net = firemodule(net, 64, 256, 'fire9')
        print(net.name, net.get_shape())
        net = tf.layers.average_pooling2d(net, (4,4), strides=(4,4), name='avgpool9')
        print(net.name, net.get_shape())
        net = tf.layers.conv2d(net, num_classes, (1,1))
        print(net.name, net.get_shape())
        logits = tf.squeeze(net, (1,2), name='logits')
        print(net.name, net.get_shape())
    return logits
        

In [8]:
def loss(logits, labels, total=True):
#    labels = tf.cast(labels, tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits, name='cross_entropy')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='mean_cross_entropy')
    tf.add_to_collection('losses', cross_entropy_mean)
    _loss = cross_entropy_mean
    if total:
        print("adding total loss", len(tf.get_collection('losses')))
        _loss = tf.add_n(tf.get_collection('losses'), name='total_loss')
    return _loss

In [9]:
def _add_total_loss_summary(total_loss, test=False):
    emaloss = tf.train.ExponentialMovingAverage(0.9, name='avg')
    losses = []
    if not test:
        losses = tf.get_collection('losses')

    loss_average_op = emaloss.apply(losses + [total_loss])
    for ls in losses + [total_loss]:
        tf.summary.scalar(ls.op.name + '_raw_', ls)
        tf.summary.scalar(ls.op.name + 'ema', emaloss.average(ls))
    return loss_average_op

In [10]:
def trainop(global_step, topk=1):
    num_batches_per_epoch = Num_Examples_Per_Epoch_For_Train / Flags.batch_size
    decay_steps = int(num_batches_per_epoch * Num_Epochs_Per_Decay)
    lr = tf.train.exponential_decay(Initial_Learning_Rate, global_step, decay_steps, Learning_Rate_Decay_Factor, staircase=True)
    tf.summary.scalar('learning_rate', lr)
    
    images, labels = input_batches(8)
    logits = squeeze_net(images, Num_Classes)
    totalloss = loss(logits, labels)
    predictions = tf.nn.in_top_k(logits, labels, topk)
    truecount = tf.cast(predictions, tf.float32)
    accuracy = tf.reduce_mean(truecount)
   
    loss_average_op = _add_total_loss_summary(totalloss)
    
    optimizer = tf.train.AdamOptimizer(lr)
    
#    return optimizer.minimize(totalloss), totalloss

    with tf.control_dependencies([loss_average_op]):
        gradsvars = optimizer.compute_gradients(totalloss)
        apply_grads_op = optimizer.apply_gradients(gradsvars, global_step=global_step)
    for grad, var in gradsvars:
        tf.summary.histogram(var.op.name, var)
        if grad is not None:
            tf.summary.histogram(var.op.name + '/grads', grad)
    
    ema_variables = tf.train.ExponentialMovingAverage(Moving_Average_Decay, global_step)
    vars_averages_op = ema_variables.apply(tf.trainable_variables())

    with tf.control_dependencies([apply_grads_op, vars_averages_op]):
        train_op = tf.no_op(name='train')
    
    return train_op, totalloss, accuracy
    
    

In [11]:
def evalop(topk=1):
    images, labels = input_batches(2, test=True)
    logits = squeeze_net(images, Num_Classes, reuse=True)
    evalloss = loss(logits, labels, total=False)
    predictions = tf.nn.in_top_k(logits, labels, topk)
    truecount = tf.cast(predictions, tf.float32)
    accuracy = tf.reduce_mean(truecount)
    return evalloss, accuracy
    

# Train, Eval

In [12]:
def train():
    with tf.Graph().as_default():
        global_step = tf.contrib.framework.get_or_create_global_step()
        train_op, totalloss, trainaccuracy = trainop(global_step)
        evalloss, evalaccuracy = evalop()
        class _LoggerHook(tf.train.SessionRunHook):
            def begin(self):
                self._step = -1
            def before_run(self, run_context):
                self._step += 1
                self._start_time = time.time()
                if self._step % 100 == 0:
                    return tf.train.SessionRunArgs([totalloss, trainaccuracy, evalloss, evalaccuracy])
                if self._step % 10 == 0:
                    return tf.train.SessionRunArgs([totalloss, trainaccuracy])
                else:
                    return tf.train.SessionRunArgs([totalloss])
            def after_run(self, run_context, run_values):
                duration = time.time() - self._start_time
                loss_value = run_values.results[0]
                if self._step % 10 == 0:
                    trainaccuracy = run_values.results[1]
                    num_examples_per_step = Flags.batch_size
                    examples_per_sec = num_examples_per_step / duration
                    sec_per_batch = float(duration)

                    format_str = ('%s: step %d, loss = %.2f  accuracy = %.2f (%.1f examples/sec; %.3f '
                                  'sec/batch)')
                    print (format_str % (time.time(), self._step, loss_value, trainaccuracy, 
                               examples_per_sec, sec_per_batch))
                if self._step % 100 == 0:
                    evalloss = run_values.results[2]
                    accuracy = run_values.results[3]
                    print('eval result: eval_loss: %.2f  accuracy: %.2f'  % (evalloss, accuracy))
                    print('')
            
        with tf.train.MonitoredTrainingSession(
            checkpoint_dir=Flags.train_dir, 
            hooks=[tf.train.StopAtStepHook(last_step=Flags.max_steps), tf.train.NanTensorHook(totalloss), _LoggerHook()], 
            config=tf.ConfigProto(log_device_placement=Flags.log_device_placement)) as mon_sess:
            while not mon_sess.should_stop():
                mon_sess.run(train_op)

In [ ]:
train()

squeezenet/conv1/BiasAdd:0 (128, 32, 32, 96)
squeezenet/maxpool1/MaxPool:0 (128, 16, 16, 96)
squeezenet/fire2/concat:0 (128, 16, 16, 128)
squeezenet/fire3/concat:0 (128, 16, 16, 128)
squeezenet/fire4/concat:0 (128, 16, 16, 256)
squeezenet/maxpool4/MaxPool:0 (128, 8, 8, 256)
squeezenet/fire5/concat:0 (128, 8, 8, 256)
squeezenet/fire6/concat:0 (128, 8, 8, 384)
squeezenet/fire7/concat:0 (128, 8, 8, 384)
squeezenet/fire8/concat:0 (128, 8, 8, 512)
squeezenet/maxpool8/MaxPool:0 (128, 4, 4, 512)
squeezenet/fire9/concat:0 (128, 4, 4, 512)
squeezenet/avgpool9/AvgPool:0 (128, 1, 1, 512)
squeezenet/conv2d/BiasAdd:0 (128, 1, 1, 10)
squeezenet/conv2d/BiasAdd:0 (128, 1, 1, 10)
adding total loss 1
squeezenet_1/conv1/BiasAdd:0 (256, 32, 32, 96)
squeezenet_1/maxpool1/MaxPool:0 (256, 16, 16, 96)
squeezenet_1/fire2/concat:0 (256, 16, 16, 128)
squeezenet_1/fire3/concat:0 (256, 16, 16, 128)
squeezenet_1/fire4/concat:0 (256, 16, 16, 256)
squeezenet_1/maxpool4/MaxPool:0 (256, 8, 8, 256)
squeezenet_1/fire5/co

1497529382.388494: step 630, loss = 1.57  accuracy = 0.45 (1523.7 examples/sec; 0.084 sec/batch)
1497529383.2345421: step 640, loss = 1.58  accuracy = 0.41 (1523.7 examples/sec; 0.084 sec/batch)
1497529384.0795906: step 650, loss = 1.80  accuracy = 0.36 (1505.8 examples/sec; 0.085 sec/batch)
1497529384.925639: step 660, loss = 1.56  accuracy = 0.44 (1505.8 examples/sec; 0.085 sec/batch)
1497529385.7696874: step 670, loss = 1.79  accuracy = 0.37 (1454.5 examples/sec; 0.088 sec/batch)
1497529386.6177359: step 680, loss = 1.64  accuracy = 0.40 (1438.1 examples/sec; 0.089 sec/batch)
1497529387.462784: step 690, loss = 1.64  accuracy = 0.41 (1471.2 examples/sec; 0.087 sec/batch)
INFO:tensorflow:global_step/sec: 11.552
1497529388.4558408: step 700, loss = 1.61  accuracy = 0.36 (547.0 examples/sec; 0.234 sec/batch)
eval result: eval_loss: 1.65  accuracy: 0.41

1497529389.31389: step 710, loss = 1.71  accuracy = 0.33 (1580.2 examples/sec; 0.081 sec/batch)
1497529390.1589384: step 720, loss = 1

1497529449.0023038: step 1400, loss = 1.48  accuracy = 0.46 (558.9 examples/sec; 0.229 sec/batch)
eval result: eval_loss: 1.46  accuracy: 0.46

1497529449.8733537: step 1410, loss = 1.39  accuracy = 0.48 (1523.7 examples/sec; 0.084 sec/batch)
1497529450.7264025: step 1420, loss = 1.34  accuracy = 0.50 (1488.3 examples/sec; 0.086 sec/batch)
1497529451.5774512: step 1430, loss = 1.47  accuracy = 0.52 (1523.7 examples/sec; 0.084 sec/batch)
1497529452.4285: step 1440, loss = 1.47  accuracy = 0.43 (1523.7 examples/sec; 0.084 sec/batch)
1497529453.2765484: step 1450, loss = 1.53  accuracy = 0.48 (1471.2 examples/sec; 0.087 sec/batch)
1497529454.1225967: step 1460, loss = 1.44  accuracy = 0.51 (1438.1 examples/sec; 0.089 sec/batch)
1497529454.965645: step 1470, loss = 1.59  accuracy = 0.42 (1488.3 examples/sec; 0.086 sec/batch)
1497529455.8136935: step 1480, loss = 1.42  accuracy = 0.50 (1542.1 examples/sec; 0.083 sec/batch)
1497529456.6637423: step 1490, loss = 1.61  accuracy = 0.32 (1523.7 

1497529515.4901068: step 2170, loss = 1.54  accuracy = 0.37 (1505.8 examples/sec; 0.085 sec/batch)
1497529516.3401556: step 2180, loss = 1.25  accuracy = 0.53 (1505.8 examples/sec; 0.085 sec/batch)
1497529517.1922042: step 2190, loss = 1.45  accuracy = 0.47 (1471.2 examples/sec; 0.087 sec/batch)
INFO:tensorflow:global_step/sec: 11.5614
1497529518.1842608: step 2200, loss = 1.41  accuracy = 0.45 (566.3 examples/sec; 0.226 sec/batch)
eval result: eval_loss: 1.43  accuracy: 0.50

1497529519.0653112: step 2210, loss = 1.53  accuracy = 0.48 (1376.3 examples/sec; 0.093 sec/batch)
1497529519.9103596: step 2220, loss = 1.52  accuracy = 0.41 (1560.9 examples/sec; 0.082 sec/batch)
1497529520.7684088: step 2230, loss = 1.34  accuracy = 0.49 (1454.5 examples/sec; 0.088 sec/batch)
1497529521.6134572: step 2240, loss = 1.25  accuracy = 0.56 (1542.1 examples/sec; 0.083 sec/batch)
1497529522.469506: step 2250, loss = 1.39  accuracy = 0.52 (1454.5 examples/sec; 0.088 sec/batch)
1497529523.3195548: step

1497529582.4159348: step 2940, loss = 1.30  accuracy = 0.52 (1488.3 examples/sec; 0.086 sec/batch)
1497529583.276984: step 2950, loss = 1.36  accuracy = 0.56 (1488.3 examples/sec; 0.086 sec/batch)
1497529584.1250324: step 2960, loss = 1.37  accuracy = 0.48 (1620.2 examples/sec; 0.079 sec/batch)
1497529584.9790814: step 2970, loss = 1.30  accuracy = 0.52 (1523.7 examples/sec; 0.084 sec/batch)
1497529585.8411307: step 2980, loss = 1.44  accuracy = 0.47 (1454.5 examples/sec; 0.088 sec/batch)
1497529586.6891792: step 2990, loss = 1.21  accuracy = 0.59 (1523.7 examples/sec; 0.084 sec/batch)
INFO:tensorflow:global_step/sec: 11.5134
1497529587.684236: step 3000, loss = 1.21  accuracy = 0.55 (554.1 examples/sec; 0.231 sec/batch)
eval result: eval_loss: 1.34  accuracy: 0.49

1497529588.5462854: step 3010, loss = 1.27  accuracy = 0.57 (1505.8 examples/sec; 0.085 sec/batch)
1497529589.3943338: step 3020, loss = 1.24  accuracy = 0.57 (1523.7 examples/sec; 0.084 sec/batch)
1497529590.2393823: step 

1497529649.1427512: step 3710, loss = 0.98  accuracy = 0.66 (1488.3 examples/sec; 0.086 sec/batch)
1497529649.9928: step 3720, loss = 1.50  accuracy = 0.48 (1505.8 examples/sec; 0.085 sec/batch)
1497529650.8418486: step 3730, loss = 1.05  accuracy = 0.64 (1580.2 examples/sec; 0.081 sec/batch)
1497529651.6968975: step 3740, loss = 1.15  accuracy = 0.61 (1488.3 examples/sec; 0.086 sec/batch)
1497529652.5429459: step 3750, loss = 1.13  accuracy = 0.64 (1542.1 examples/sec; 0.083 sec/batch)
1497529653.382994: step 3760, loss = 1.26  accuracy = 0.49 (1542.1 examples/sec; 0.083 sec/batch)
1497529654.2200418: step 3770, loss = 1.29  accuracy = 0.56 (1620.2 examples/sec; 0.079 sec/batch)
1497529655.0710905: step 3780, loss = 1.19  accuracy = 0.59 (1542.1 examples/sec; 0.083 sec/batch)
1497529655.9261394: step 3790, loss = 1.13  accuracy = 0.61 (1454.5 examples/sec; 0.088 sec/batch)
INFO:tensorflow:global_step/sec: 11.5574
1497529656.917196: step 3800, loss = 1.23  accuracy = 0.62 (566.3 exampl

1497529715.644555: step 4480, loss = 1.16  accuracy = 0.59 (1488.3 examples/sec; 0.086 sec/batch)
1497529716.4966037: step 4490, loss = 1.24  accuracy = 0.54 (1438.1 examples/sec; 0.089 sec/batch)
INFO:tensorflow:global_step/sec: 11.5614
1497529717.5026612: step 4500, loss = 1.26  accuracy = 0.53 (542.3 examples/sec; 0.236 sec/batch)
eval result: eval_loss: 1.18  accuracy: 0.60

1497529718.3707108: step 4510, loss = 1.06  accuracy = 0.63 (1471.2 examples/sec; 0.087 sec/batch)
1497529719.2187593: step 4520, loss = 1.22  accuracy = 0.57 (1406.5 examples/sec; 0.091 sec/batch)
1497529720.0728083: step 4530, loss = 1.26  accuracy = 0.60 (1438.1 examples/sec; 0.089 sec/batch)
1497529720.9488585: step 4540, loss = 1.19  accuracy = 0.59 (1542.1 examples/sec; 0.083 sec/batch)
1497529721.796907: step 4550, loss = 1.38  accuracy = 0.51 (1488.3 examples/sec; 0.086 sec/batch)
1497529722.6469555: step 4560, loss = 1.18  accuracy = 0.62 (1523.7 examples/sec; 0.084 sec/batch)
1497529723.4880035: step 

1497529782.2333636: step 5250, loss = 1.05  accuracy = 0.66 (1542.1 examples/sec; 0.083 sec/batch)
1497529783.0824122: step 5260, loss = 1.29  accuracy = 0.54 (1523.7 examples/sec; 0.084 sec/batch)
1497529783.9294608: step 5270, loss = 1.29  accuracy = 0.55 (1523.7 examples/sec; 0.084 sec/batch)
1497529784.776509: step 5280, loss = 1.13  accuracy = 0.61 (1438.1 examples/sec; 0.089 sec/batch)
1497529785.6205573: step 5290, loss = 1.14  accuracy = 0.53 (1488.3 examples/sec; 0.086 sec/batch)
INFO:tensorflow:global_step/sec: 11.5976
1497529786.6026137: step 5300, loss = 1.18  accuracy = 0.55 (574.0 examples/sec; 0.223 sec/batch)
eval result: eval_loss: 1.32  accuracy: 0.52

1497529787.4786637: step 5310, loss = 1.17  accuracy = 0.55 (1580.2 examples/sec; 0.081 sec/batch)
1497529788.3357127: step 5320, loss = 0.96  accuracy = 0.66 (1505.8 examples/sec; 0.085 sec/batch)
1497529789.1827612: step 5330, loss = 0.95  accuracy = 0.68 (1523.7 examples/sec; 0.084 sec/batch)
1497529790.03381: step 5

1497529848.8791757: step 6020, loss = 1.00  accuracy = 0.66 (1505.8 examples/sec; 0.085 sec/batch)
1497529849.7282243: step 6030, loss = 1.23  accuracy = 0.59 (1471.2 examples/sec; 0.087 sec/batch)
1497529850.5792727: step 6040, loss = 1.11  accuracy = 0.59 (1523.7 examples/sec; 0.084 sec/batch)
1497529851.4273212: step 6050, loss = 1.04  accuracy = 0.62 (1523.7 examples/sec; 0.084 sec/batch)
1497529852.27537: step 6060, loss = 1.09  accuracy = 0.61 (1505.8 examples/sec; 0.085 sec/batch)
1497529853.1294186: step 6070, loss = 1.00  accuracy = 0.66 (1542.1 examples/sec; 0.083 sec/batch)
1497529853.973467: step 6080, loss = 0.90  accuracy = 0.69 (1580.2 examples/sec; 0.081 sec/batch)
1497529854.8235157: step 6090, loss = 1.00  accuracy = 0.69 (1454.5 examples/sec; 0.088 sec/batch)
INFO:tensorflow:global_step/sec: 11.4976
1497529855.8165724: step 6100, loss = 0.94  accuracy = 0.66 (556.5 examples/sec; 0.230 sec/batch)
eval result: eval_loss: 1.15  accuracy: 0.59

1497529856.6756215: step 6

1497529915.4319823: step 6790, loss = 1.09  accuracy = 0.62 (1488.3 examples/sec; 0.086 sec/batch)
INFO:tensorflow:global_step/sec: 11.5855
1497529916.409038: step 6800, loss = 1.14  accuracy = 0.63 (576.5 examples/sec; 0.222 sec/batch)
eval result: eval_loss: 1.16  accuracy: 0.57

1497529917.282088: step 6810, loss = 1.16  accuracy = 0.60 (1488.3 examples/sec; 0.086 sec/batch)
1497529918.1291366: step 6820, loss = 1.10  accuracy = 0.62 (1523.7 examples/sec; 0.084 sec/batch)
1497529918.975185: step 6830, loss = 0.89  accuracy = 0.65 (1471.2 examples/sec; 0.087 sec/batch)
1497529919.820233: step 6840, loss = 0.94  accuracy = 0.69 (1454.5 examples/sec; 0.088 sec/batch)
1497529920.6782823: step 6850, loss = 1.08  accuracy = 0.58 (1488.3 examples/sec; 0.086 sec/batch)
1497529921.532331: step 6860, loss = 1.01  accuracy = 0.63 (1488.3 examples/sec; 0.086 sec/batch)
1497529922.3843799: step 6870, loss = 0.78  accuracy = 0.72 (1454.5 examples/sec; 0.088 sec/batch)
1497529923.2404287: step 688

In [16]:
with tf.variable_scope("foo"):
    v = tf.get_variable('v1', [], tf.float32)

ValueError: Variable foo/v1 already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "C:\Program Files\Anaconda2\envs\python3\lib\site-packages\tensorflow\python\framework\ops.py", line 1228, in __init__
    self._traceback = _extract_stack()
  File "C:\Program Files\Anaconda2\envs\python3\lib\site-packages\tensorflow\python\framework\ops.py", line 2336, in create_op
    original_op=self._default_original_op, op_def=op_def)
  File "C:\Program Files\Anaconda2\envs\python3\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 768, in apply_op
    op_def=op_def)


In [21]:
with tf.variable_scope("foo", reuse=True):
    v2= tf.get_variable("v1",[])
    print(v2.name)

foo/v1:0


In [26]:
tf.global_variables()

[]

In [20]:
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

[]

In [27]:
train_op

<tf.Operation 'GradientDescent' type=NoOp>

In [28]:
tf.trainable_variables()

[]

In [29]:
sess = tf.Session()

In [30]:
sess.run(train_op)

RuntimeError: The Session graph is empty.  Add operations to the graph before calling run().